In [1]:

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import r2_score
from sklearn.impute import KNNImputer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import RidgeCV
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso

In [3]:
# Load the dataset
file_path = 'data/obp.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()

,Name,playerid,birth_date,PA_21,OBP_21,PA_20,OBP_20,PA_19,OBP_19,PA_18,OBP_18,PA_17,OBP_17,PA_16,OBP_16
0,Trayce Thompson,9952,1991-03-15,35,0.400,NaN,NaN,NaN,NaN,137.0,0.162,55.0,0.218,262.0,0.302
1,Mike Trout,10155,1991-08-07,146,0.466,241.0,0.390,600.0,0.438,608.0,0.460,507.0,0.442,681.0,0.441
2,Bryce Harper,11579,1992-10-16,599,0.429,244.0,0.420,682.0,0.372,695.0,0.393,492.0,0.413,627.0,0.373
3,Chris Owings,10030,1991-08-12,50,0.420,44.0,0.318,196.0,0.209,309.0,0.272,386.0,0.299,466.0,0.315
4,Nick Fortes,21538,1996-11-11,34,0.353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Baseline Model (Linear Regression)


In [4]:

# Filtering relevant columns for the calculation
obp_columns = [col for col in data.columns if 'OBP' in col and '21' not in col]
pa_columns = [col for col in data.columns if 'PA' in col and '21' not in col]


features = data[obp_columns + pa_columns].fillna(0)  # Filling missing values with 0

# Target variable: OBP in 2021
target = data['OBP_21'].fillna(0)  # Filling missing values with 0

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Creating and training the linear regression model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predicting OBP for 2021
y_pred = linear_model.predict(X_test)

# Calculating the R-squared value
r_squared_linear = r2_score(y_test, y_pred)
r_squared_linear

0.1254860670324529

## Ridge Regression Next Step

In [5]:
# Load the CSV file to analyze its contents
file_path = ' data/fg.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()

,Season,Name,Team,G,PA,HR,R,RBI,SB,BB%,...,maxEV,HardHit%,CStr%,CSW%,xBA,xSLG,xwOBA.1,NameASCII,PlayerId,MLBAMID
0,2020,Juan Soto,WSN,47,196,13,39,37,6,0.209184,...,113.291,0.5159,0.1850,0.2503,NaN,NaN,NaN,Juan Soto,20123,665742
1,2018,Mike Trout,LAA,140,608,39,101,79,24,0.200658,...,118.025,0.4602,0.2013,0.2611,NaN,NaN,NaN,Mike Trout,10155,545361
2,2018,Luke Voit,- - -,47,161,15,30,36,0,0.105590,...,112.270,0.5400,0.1237,0.2754,NaN,NaN,NaN,Luke Voit,14811,572228
3,2020,Freddie Freeman,ATL,60,262,13,51,53,2,0.171756,...,109.342,0.5424,0.1038,0.1910,NaN,NaN,NaN,Freddie Freeman,5361,518692
4,2018,Mookie Betts,BOS,136,614,32,129,80,30,0.131922,...,110.635,0.5000,0.2196,0.2699,NaN,NaN,NaN,Mookie Betts,13611,605141


In [6]:
## Focusing on Rate Bate stats, this should be considered EDA

predictor_vars = ['Barrel%', 'CStr%', 'SwStr%', 'O-Swing%', 'Z-Swing%', 'O-Contact%', 'Z-Contact%', 'FB%', 'GB%', 'LA', 'EV']

data_prior = data[data['Season'] < 2021]


player_stats_prior = data_prior.groupby('Name')[predictor_vars].mean().reset_index()
obp_target_prior = data_prior.groupby('Name')['OBP'].mean()


# Now we will create the Ridge regression model and train it on the data prior to 2021
ridge_model = RidgeCV(alphas=np.logspace(-6, 6, 13), cv=5)

# We need to join our player stats with their OBP targets
training_data = player_stats_prior.merge(obp_target_prior, on='Name')

# Selecting features and target for training
X_train = training_data[predictor_vars]
y_train = training_data['OBP']

# Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Fit the model
ridge_model.fit(X_train_scaled, y_train)

# We can use the trained model to predict Juan Soto's OBP for 2021
# First, we need to get the average stats for Juan Soto from prior years
soto_stats = player_stats_prior[player_stats_prior['Name'] == 'Juan Soto'][predictor_vars]

# If Juan Soto is in the dataset, predict his 2021 OBP
if soto_in_data:
    soto_stats_scaled = scaler.transform(soto_stats)
    soto_pred_2021 = ridge_model.predict(soto_stats_scaled)
else:
    soto_pred_2021 = [np.nan]

# Calculate R-squared on the training data for reference
ridge_r2 = ridge_model.score(X_train_scaled, y_train)

# Extracting feature importance from the model
feature_importance = pd.DataFrame({
    'Feature': predictor_vars,
    'Coefficient': ridge_model.coef_
}).sort_values(by='Coefficient', ascending=False)

ridge_r2, soto_pred_2021, feature_importance



(0.5661245405221307,
 array([0.38316599]),
        Feature  Coefficient
 0      Barrel%     0.015144
 5   O-Contact%     0.014387
 6   Z-Contact%     0.006376
 10          EV     0.005496
 2       SwStr%     0.003239
 9           LA     0.000549
 4     Z-Swing%    -0.006704
 1        CStr%    -0.013093
 3     O-Swing%    -0.017027
 8          GB%    -0.021347
 7          FB%    -0.027681)

In [7]:

# Predicting OBP for all players based on their average stats from prior years
player_stats_prior_scaled = scaler.transform(player_stats_prior[predictor_vars])
player_stats_prior['Projected_OBP'] = ridge_model.predict(player_stats_prior_scaled)

# Next, we'll retrieve the actual OBP for 2021.
obp_actual_2021 = data[data['Season'] == 2021][['Name', 'OBP']].set_index('Name')

# Merge the projected OBP with the actual OBP for 2021
obp_comparison = player_stats_prior.merge(obp_actual_2021, on='Name', how='left')
obp_comparison.rename(columns={'OBP': 'Actual_OBP'}, inplace=True)

# Sort the DataFrame by projected OBP from highest to lowest
obp_comparison_sorted = obp_comparison.sort_values(by='Projected_OBP', ascending=False)

# Select only the columns we want to display
obp_comparison_final = obp_comparison_sorted[['Name', 'Projected_OBP', 'Actual_OBP']]

obp_comparison_final.reset_index(drop=True, inplace=True)


obp_comparison_filtered = obp_comparison_final.dropna(subset=['Actual_OBP'])

obp_comparison_filtered

,Name,Projected_OBP,Actual_OBP
1,Freddie Freeman,0.393870,0.392806
2,Joey Votto,0.391729,0.375235
3,Yordan Alvarez,0.389846,0.346154
6,Luis Arraez,0.384691,0.356994
7,Aaron Judge,0.383622,0.372828
...,...,...,...
625,Lewis Brinson,0.277371,0.262976
629,Austin Hedges,0.276444,0.219672
630,Tim Locastro,0.275449,0.262821
635,Francisco Mejía,0.273830,0.322464


## Historical Player Comparison Modelling
- Finding historical comps for a player (using data after 2002, as we only have access to plate discipline metrics after then)
- Using PCA - originally tried to use Mahalanobis Distance, but it was computationally expensive, and I'm generally less familiar with it (I'm pretty sure ZIPS utilizes Mahalanobis Distance). 
- After we find the comparisons, we get the historical comps following season OBP.
- We are going to use a weighted average based on distance to the target player, and that weighted OBP will be our projected OBP for the player.

In [14]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Load your dataset
file_path = ' data/fg2.csv'  # Replace with your dataset file path
data = pd.read_csv(file_path)

# Preprocess the data
relevant_columns = data.columns.drop(['Season', 'Name', 'Team', 'NameASCII', 'PlayerId', 'MLBAMID'])
numeric_data = data[relevant_columns]
scaler = StandardScaler()
standardized_data = scaler.fit_transform(numeric_data)
imputer = SimpleImputer(strategy='median')
standardized_data_imputed = imputer.fit_transform(standardized_data)
pca = PCA(n_components=0.95)
pca_data = pca.fit_transform(standardized_data_imputed)

# Function to calculate Euclidean Distance
def calculate_euclidean_distance(target_index, pca_data):
    distances = []
    target = pca_data[target_index]
    for index, row in enumerate(pca_data):
        distance = euclidean(target, row)
        distances.append((index, distance))
    return distances

# Function to find similar player seasons
def find_similar_player_seasons(name, season, data, pca_data, num_similar):
    target_index = data[(data['Name'] == name) & (data['Season'] == season)].index
    if target_index.empty:
        return "Player-season not found in the dataset."
    
    distances = calculate_euclidean_distance(target_index[0], pca_data)
    result = []
    for index, distance in sorted(distances, key=lambda x: x[1]):
        if data.iloc[index]['Name'] != name:  # Exclude target player
            result.append(data.iloc[index].to_dict())
            result[-1]['Distance'] = distance
            if len(result) == num_similar:
                break

    return result

# Function to get next season OBP
def get_next_season_obp(player_name, season, data):
    next_season = season + 1
    next_season_data = data[(data['Name'] == player_name) & (data['Season'] == next_season)]
    if not next_season_data.empty:
        return next_season_data.iloc[0]['OBP+']
    else:
        return None

# Function for weighted prediction
def weighted_prediction(similar_seasons_df, data):
    predictions = []
    total_weight = 0
    for _, row in similar_seasons_df.iterrows():
        next_season_obp = get_next_season_obp(row['Name'], row['Season'], data)
        if next_season_obp is not None:
            weight = 1 / (row['Distance'] + 1)
            weighted_obp = weight * next_season_obp
            predictions.append(weighted_obp)
            total_weight += weight
    if total_weight > 0:
        return sum(predictions) / total_weight
    else:
        return None

# Predicting 2017 OBP for all players in the 2016 season
players_2014 = data[data['Season'] == 2014]['Name'].unique()
predictions_2015 = []
actuals_2015 = []

for player in players_2014:
    similar_seasons = find_similar_player_seasons(player, 2014, data, pca_data, 30)
    if isinstance(similar_seasons, list):  # Ensure that similar seasons were found
        similar_seasons_df = pd.DataFrame(similar_seasons)
        predicted_obp = weighted_prediction(similar_seasons_df, data)
        actual_obp = get_next_season_obp(player, 2014, data)
        if predicted_obp is not None and actual_obp is not None:
            predictions_2015.append(predicted_obp)
            actuals_2015.append(actual_obp)

# Calculating R² value
r_squared = r2_score(actuals_2015, predictions_2015)
print(f"R² value for the 2017 OBP predictions: {r_squared}")


R² value for the 2017 OBP predictions: 0.4468370402318844


### Historical Comparison Model, number of comparisons evaluation:

**2015 -> Predicting 2016**

25 players - R² value for the 2017 OBP predictions: 0.42544461189140204

30 players - R² value for the 2017 OBP predictions: 0.43040822755165087

35 pkayers - R² value for the 2017 OBP predictions: 0.42775861604286436


**2016 -> Predicting 2017**

10 players - R² value for the 2017 OBP predictions: 0.38242495810244903

20 players - R² value for the 2017 OBP predictions: 0.44997848901994675

25 players - R² value for the 2017 OBP predictions: 0.46119761219361532

30 players-  R² value for the 2017 OBP predictions: 0.4736488492964589

**Seems like 30 is our best bet.**

In [29]:
file_path2 = ' data/fg2.csv'
data2 = pd.read_csv(file_path2)

def add_historical_comp_OBP(data):
    # Preprocess the data
    relevant_columns = data.columns.drop(['Season', 'Name', 'Team', 'NameASCII', 'PlayerId', 'MLBAMID'])
    numeric_data = data[relevant_columns]
    scaler = StandardScaler()
    standardized_data = scaler.fit_transform(numeric_data)
    imputer = SimpleImputer(strategy='median')
    standardized_data_imputed = imputer.fit_transform(standardized_data)
    pca = PCA(n_components=0.95)
    pca_data = pca.fit_transform(standardized_data_imputed)

    # Predicting OBP for each player in each season
    historical_comp_OBP = []
    for index, row in data.iterrows():
        player = row['Name']
        season = row['Season']
        similar_seasons = find_similar_player_seasons(player, season, data, pca_data, 30)
        if isinstance(similar_seasons, list):  # Ensure that similar seasons were found
            similar_seasons_df = pd.DataFrame(similar_seasons)
            predicted_obp = weighted_prediction(similar_seasons_df, data)
            historical_comp_OBP.append(predicted_obp)
        else:
            historical_comp_OBP.append(None)
    
    data['historical_comp_OBP'] = historical_comp_OBP
    return data

# Add the historical_comp_OBP to the data
data2= add_historical_comp_OBP(data2)
data2.head()

,Season,Name,Team,PA,O-Swing%,Swing%,Pull%,CStr%,CSW%,BB%+,...,GB%+,FB%+,HR/FB%+,Pull%+,Hard%+,OBP+,NameASCII,PlayerId,MLBAMID,historical_comp_OBP
0,2004,Barry Bonds,SFG,617,0.1245,0.3427,0.4806,0.1241,0.1641,415.634135,...,79.786384,130.185332,249.249594,111.829945,170.644270,178.552168,Barry Bonds,1109,111188,121.046956
1,2002,Barry Bonds,SFG,612,0.1131,0.3662,0.4469,0.1272,0.1906,346.706378,...,69.571669,135.231765,249.230249,104.754659,170.730231,171.204358,Barry Bonds,1109,111188,121.215342
2,2003,Barry Bonds,SFG,550,0.1990,0.4020,0.4760,0.1347,0.2230,298.321654,...,69.262974,133.829874,255.729243,108.730180,160.597384,155.375966,Barry Bonds,1109,111188,119.007755
3,2020,Juan Soto,WSN,196,0.2100,0.3603,0.3254,0.1850,0.2503,225.616682,...,120.468030,80.711615,239.993571,80.010037,123.897265,150.635981,Juan Soto,20123,665742,119.236936
4,2021,Mike Trout,LAA,146,0.2210,0.3951,0.4868,0.1738,0.2787,218.196939,...,99.240267,92.319908,225.885584,120.886086,118.615131,147.139811,Mike Trout,10155,545361,107.094977


In [46]:
# Load the dataset
file_path = ' data/fg.csv'
data = pd.read_csv(file_path)

# Loading in our quality of contact data - which was calculated in R
file_path_qoc = ' data/p_QOC.csv'
p_QOC = pd.read_csv(file_path_qoc)

data = pd.merge(data,  p_QOC, on=['Season', 'MLBAMID'], 
                          how='left')
data.head()

,Season,Name,Team,G,PA,HR,R,RBI,SB,BB%,...,MLBAMID,player_name,BIP,xHR,xBACON,xSLGCON,EV81,EV96,best_speed,hard_hit_rate
0,2020,Juan Soto,WSN,47,196,13,39,37,6,0.209184,...,665742,"Soto, Juan",126.0,15.233077,0.408238,0.787231,102.680,109.30,99.911429,0.341232
1,2018,Mike Trout,LAA,140,608,39,101,79,24,0.200658,...,545361,"Trout, Mike",339.0,36.123617,0.392157,0.723639,101.600,111.20,98.270915,0.305419
2,2018,Luke Voit,- - -,47,161,15,30,36,0,0.105590,...,572228,"Voit, Luke",96.0,14.272941,0.432995,0.892973,100.900,108.00,98.809639,0.341317
3,2020,Freddie Freeman,ATL,60,262,13,51,53,2,0.171756,...,518692,"Freeman, Freddie",176.0,18.747398,0.425827,0.764571,99.400,106.00,97.045977,0.308571
4,2018,Mookie Betts,BOS,136,614,32,129,80,30,0.131922,...,605141,"Betts, Mookie",404.0,40.233510,0.393293,0.704607,100.945,106.24,99.236646,0.379257


In [50]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error


# Step 2: Generate Lagged Features
def generate_lags(df, variables, lags=[1, 2, 3]):
    lagged_df = pd.DataFrame()
    for var in variables:
        for lag in lags:
            lag_name = f'{var}_lag{lag}'
            lagged_df[lag_name] = df.groupby('Name')[var].shift(lag)
            if var == 'PA':  # Special handling for 'PA'
                lagged_df[lag_name].fillna(0, inplace=True)
            else:
                lagged_df[lag_name].fillna(df.groupby('Name')[var].transform('mean'), inplace=True)
    return lagged_df

predictors = ['PA', 'OBP', 'Barrel%', 'CStr%', 'SwStr%', 'O-Swing%', 'Z-Swing%', 
              'O-Contact%', 'Z-Contact%', 'FB%', 'GB%', 'LA', 'EV', 'BABIP', 'K%', 'BB%', 'ISO', 'BIP', "xHR", 'xBACON', 'xSLGCON', 'EV81', 'EV96','best_speed', 'hard_hit_rate']

lagged_features = generate_lags(data, predictors)

# Step 3: Prepare the Data for Modeling
data_with_lags = pd.concat([data.reset_index(drop=True), lagged_features.reset_index(drop=True)], axis=1)
data_with_lags = data_with_lags.dropna(subset=['OBP'])


# Custom function to calculate weighted mean and standard deviation
def weighted_stats(data, weights, features):
    weighted_means = {}
    weighted_stds = {}
    for feature in features:
        # Ensuring the sum of weights is not zero
        if np.sum(weights) == 0:
            raise ValueError("Sum of weights for feature {} is zero".format(feature))
        weighted_mean = np.average(data[feature], weights=weights)
        weighted_std = np.sqrt(np.average((data[feature] - weighted_mean)**2, weights=weights))
        weighted_means[feature] = weighted_mean
        weighted_stds[feature] = weighted_std
    return weighted_means, weighted_stds

# Custom function to scale features using weighted statistics
def scale_features(data, features, means, stds):
    scaled_data = data.copy()
    for feature in features:
        scaled_data[feature] = (data[feature] - means[feature]) / stds[feature]
    return scaled_data

# Time-Series Cross-Validation
data_with_lags = pd.merge(data_with_lags, 
                          data2[['Season', 'MLBAMID', 'historical_comp_OBP']], 
                          on=['Season', 'MLBAMID'], 
                          how='left')

unique_seasons = sorted(data_with_lags['Season'].unique())
mse_scores = []
r2_scores = []

for season in unique_seasons:
    if season >= 2016:
        train = data_with_lags[data_with_lags['Season'] < season]
        test = data_with_lags[data_with_lags['Season'] == season]

        X_train = train[[col for col in train.columns if 'lag' in col]]
        y_train = train['OBP']
        X_test = test[[col for col in test.columns if 'lag' in col]]
        y_test = test['OBP']

        # Filter out rows where all 'PA' lag features are zero
        filtered_X_train = X_train[(X_train['PA_lag1'] != 0) | (X_train['PA_lag2'] != 0) | (X_train['PA_lag3'] != 0)]

        # Calculate weighted mean and standard deviation for scaling
        try:
            weighted_means, weighted_stds = weighted_stats(filtered_X_train, filtered_X_train['PA_lag1'], X_train.columns)
        except ValueError as e:
            print(f"Skipping season {season} due to error: {e}")
            continue

        # Scale the features
        X_train_scaled = scale_features(X_train, X_train.columns, weighted_means, weighted_stds)
        X_test_scaled = scale_features(X_test, X_test.columns, weighted_means, weighted_stds)

        # Model fitting and prediction
        model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.05, max_depth= 5, random_state=42)
        model.fit(X_train_scaled, y_train)
        predictions = model.predict(X_test_scaled)

        # Calculate Mean Squared Error
        mse = mean_squared_error(y_test, predictions)
        r2 = r2_score(y_test, predictions)
        mse_scores.append((season, mse))
        r2_scores.append((season, r2))

# Print the MSE for each season and the average MSE
for i in range(len(mse_scores)):
    print(f"Season {mse_scores[i][0]}: MSE = {mse_scores[i][1]}, R^2 = {r2_scores[i][1]}")

average_mse = sum([score[1] for score in mse_scores]) / len(mse_scores)
print(f'\nAverage MSE: {average_mse}')


Skipping season 2016 due to error: Sum of weights for feature PA_lag1 is zero
Season 2017: MSE = 0.00048064871880251916, R^2 = 0.6608596452222726
Season 2018: MSE = 0.00034045537734884334, R^2 = 0.7526209851290233
Season 2019: MSE = 0.00037179299043849686, R^2 = 0.7196961046270739
Season 2020: MSE = 0.0005944465793528128, R^2 = 0.6570354777516751
Season 2021: MSE = 0.0004954159396061896, R^2 = 0.6534993501843058

Average MSE: 0.0004565519211097724


Tried around 8 different hyperparamter combos. More extensive tuning would be beneficial in the future:

This was the one that consistently performed well, even when features were changing
#### `N_Estimators` - 100, `learning_rate` - 0.05, `max_depth` - 5

Season 2017: MSE = 0.00047260656520671556, R^2 = 0.6665340987617574

Season 2018: MSE = 0.0003401232969845588, R^2 = 0.7528622787576169

Season 2019: MSE = 0.00036392823298272585, R^2 = 0.725625539037378

Season 2020: MSE = 0.0005902717249706845, R^2 = 0.6594441499324147

Season 2021: MSE = 0.0004962493597545356, R^2 = 0.6529164447913125

Average MSE: 0.00045263583597984406

### Comparing with Steamer 
Now, we will compare our results with Steamer. 

It's important to consider that Steamer isn't tailored specifically to OBP. We have based every aspect of our model with respect to predicting OBP, rather than the overall production of a player.  

In [54]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

# Function to load CSV files
def load_csv(file_path):
    return pd.read_csv(file_path)

# Function to prepare and merge the datasets
def prepare_and_merge(fg_data, steamer_data, season):
    fg_season = fg_data[fg_data['Season'] == season]
    steamer_season = steamer_data[steamer_data['Season'] == season]
    merged = pd.merge(fg_season, steamer_season,  on=['Season', 'MLBAMID'], how='left')
    return merged

# Function to calculate MSE and R^2
def calculate_metrics(merged_data):
    actual_obp = merged_data['OBP_x']
    projected_obp = merged_data['OBP_y']
    mse = mean_squared_error(actual_obp, projected_obp)
    r2 = r2_score(actual_obp, projected_obp)
    return mse, r2

# Load the datasets
fg_data = load_csv(' data/fg.csv')
steamer_2019 = load_csv(' data/steamer_hitters_2019_preseason_final.csv')
steamer_2020 = load_csv(' data/steamer_hitters_2020_preseason_final.csv')
steamer_2021 = load_csv(' data/steamer_hitters_2021_preseason_final.csv')

steamer_2019 = steamer_2019.rename(columns={'proj_season': 'Season'})
steamer_2020 = steamer_2020.rename(columns={'proj_season': 'Season'})
steamer_2021 = steamer_2021.rename(columns={'proj_season': 'Season'})

steamer_2019 = steamer_2019.rename(columns={'mlbamid': 'MLBAMID'})
steamer_2020 = steamer_2020.rename(columns={'mlbamid': 'MLBAMID'})
steamer_2021 = steamer_2021.rename(columns={'mlbamid': 'MLBAMID'})

steamer_2019.head()

,MLBAMID,steamerid,Season,firstname,lastname,Team,BasicPF,position,bats,p1,...,wRAA,wRC,wRAA_adj,BattingRuns,ReplacementRuns,DEF_adj,FramingRuns,WAR,WAR/PA,frWAR
0,542194,10028,2019,Christian,Bethancourt,FA,1.000000,C,R,0.037,...,-0.023,0.097,0.000,-0.023,0.032,0.016667,-0.0051,0.00265,0.002650,0.00213
1,572008,10030,2019,Chris,Owings,KCA,1.014265,3B,R,0.039,...,-5.478,30.494,-0.513,-5.992,9.709,1.256414,0.0000,0.56977,0.001897,0.56977
2,571976,10047,2019,Wil,Myers,SDN,0.969053,LF,R,0.011,...,5.016,76.973,2.227,7.243,19.422,-6.525012,0.0000,2.31784,0.003859,2.31784
3,502205,10053,2019,Grant,Green,FA,1.000000,2b,R,0.061,...,-0.037,0.082,0.000,-0.037,0.032,0.003333,0.0000,-0.00082,-0.000820,-0.00082
4,545358,10059,2019,Max,Stassi,HOU,0.959476,C,R,0.008,...,-3.474,19.501,0.931,-2.543,6.201,4.180950,8.4642,0.68779,0.003586,1.55913


In [55]:

# Perform analysis for each season
results = {}
for season, steamer_dataset in zip([2019, 2020, 2021], [steamer_2019, steamer_2020, steamer_2021]):
    merged_data = prepare_and_merge(fg_data, steamer_dataset, season)
    mse, r2 = calculate_metrics(merged_data)
    results[season] = {'MSE': mse, 'R^2': r2}

# Display results
for season, metrics in results.items():
    print(f"Season {season}: MSE = {metrics['MSE']}, R^2 = {metrics['R^2']}")


Season 2019: MSE = 0.0009672785091703028, R^2 = 0.27074490105051074
Season 2020: MSE = 0.0016211168352848336, R^2 = 0.0647005463005722
Season 2021: MSE = 0.0010648556410646987, R^2 = 0.2552254740085159


This is suprising. Our model significantly outperforms Steamer. It outperforms Steamer to the extent that I'm nervous I've made a mistake in my modelling process. The fact that Steamer loses to the baseline model in 2020 is concerning.

That being said, even if there are minor misteps in my process, I believe that the final model I created is an effective tool at projecting year-over-year OBP. 

In the future, I would like to introduce minor league data into the model, especially for players who debuted in the last few seasons. Also, I originally wanted to introduce BIP Park Factor Bootstrapping - with respect to a players schedule in the upcoming season. However, this proved to be computationally inpractical given the time crunch. I attempted to implement a stuff+ model to reduce bias relating to players who either outperformed/underperformed based on pitch quality, but the model was surface level, and ineffective so I scrapped it. 

In conclusion, the model effectively outperforms both Steamer, and the Ridge Regression. 